In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('cumulative.csv')

# Print the shape of the dataset
print(df.shape)

# Select top of the dataset
df.head()

(9617, 49)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48
9612,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.0000,0,1,1,0,...,-166.00,4.529,0.035,-0.196,0.9030,0.2370,-0.0790,297.188750,47.093819,14.082
9613,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.4970,0,0,0,0,...,-220.00,4.444,0.056,-0.224,1.0310,0.3410,-0.1140,286.509370,47.163219,14.757
9614,10147276,K07987.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.0210,0,0,1,0,...,-236.00,4.447,0.056,-0.224,1.0410,0.3410,-0.1140,294.164890,47.176281,15.385
9615,10155286,K07988.01,NaN,CANDIDATE,CANDIDATE,0.0920,0,0,0,0,...,-128.00,2.992,0.030,-0.027,7.8240,0.2230,-1.8960,296.762880,47.145142,10.998
9616,10156110,K07989.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.0000,0,0,1,1,...,-225.00,4.385,0.054,-0.216,1.1930,0.4100,-0.1370,297.009770,47.121021,14.826
